In [1]:
import pandas as pd
from pathlib import Path

DF = pd.read_csv(Path("data/completo.csv"), low_memory=False)
print(f"✅  Datos cargados: {DF.shape[0]:,} filas × {DF.shape[1]} columnas")

✅  Datos cargados: 10,057,440 filas × 21 columnas


In [ ]:
columnas = DF.columns.tolist()
print("🗂️  Columnas:")
for c in columnas:
    print(f" • {c}")

🗂️  Columnas:
 • idListaCobro
 • idCredito
 • consecutivoCobro
 • idBanco
 • montoExigible
 • montoCobrar
 • montoCobrado
 • fechaCobroBanco
 • idRespuestaBanco
 • NombreBanco
 • DescripcionRespBanco
 • fechaCreacionLista
 • idBanco_lista
 • fechaEnvioCobro
 • idEmisora
 • NombreEmisora
 • IdBanco
 • Emisora
 • TipoEnvio
 • EmisoraCostoAceptado
 • EmisoraCostoDevuelto


In [2]:
col = "NombreBanco"           # ← cámbialo por la columna que quieras
conteo = DF[col].value_counts(dropna=False)  # incluye nulos
display(conteo.to_frame("repeticiones"))
print(f"\n🔢  Distintos en «{col}»: {conteo.size}")

,repeticiones
NombreBanco,
BBVA MEXICO,4985164
BANAMEX,3942031
SANTANDER,521543
BANORTE,381549
HSBC,186452
SCOTIABANK,16336
AZTECA,16213
AFIRME,5176
BANCOPPEL,910



🔢  Distintos en «NombreBanco»: 15


In [5]:
col = "idBanco"           # ← cámbialo por la columna que quieras
conteo = DF[col].value_counts(dropna=False)  # incluye nulos
display(conteo.to_frame("repeticiones"))
print(f"\n🔢  Distintos en «{col}»: {conteo.size}")

,repeticiones
idBanco,
12,4985164
2,3942031
14,521543
72,381549
21,186452
44,16336
127,16213
62,5176
137,910



🔢  Distintos en «idBanco»: 15


In [21]:
# Filtro de casos donde el monto a cobrar es menor al exigible
condicion = DF["montoCobrar"] == DF["montoExigible"]
casos = DF[condicion]

print(f"🔎 Casos donde montoCobrar < montoExigible: {casos.shape[0]} filas")

# Muestra los primeros 10 como ejemplo
display(casos[["idListaCobro", "idCredito", "montoExigible", "montoCobrar"]].head(10))

🔎 Casos donde montoCobrar < montoExigible: 10057440 filas


,idListaCobro,idCredito,montoExigible,montoCobrar
0,100613,83021,572.24,572.24
1,100613,87227,517.74,517.74
2,100613,103785,681.23,681.23
3,100613,110463,572.24,572.24
4,100613,113755,340.62,340.62
5,100613,118237,272.50,272.50
6,100613,136004,361.06,361.06
7,100613,137382,393.94,393.94
8,100613,137885,289.02,289.02
9,100613,149399,572.24,572.24
